# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [5]:
# import the train dataset ( 1000 first rows)
import pandas as pd
df=pd.read_csv('../data/train.csv',encoding="UTF8", sep=',', nrows=1000)
df.head(5)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


In [6]:
# prepare X and y
X = df.drop(columns=['fare_amount'])
y=df.fare_amount

In [8]:
# Hold out ( train and test dplit )
from sklearn.model_selection import train_test_split
X_train, y_train, X_test, y_test = train_test_split(X,y, test_size=0.2)

In [9]:
X_train.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
216,2009-04-08 19:11:10.0000004,2009-04-08 19:11:10 UTC,-73.998287,40.727490,-73.989854,40.735239,2
82,2013-06-14 08:27:43.0000001,2013-06-14 08:27:43 UTC,-73.953710,40.790813,-73.957015,40.777676,1
446,2010-02-26 23:56:00.000000182,2010-02-26 23:56:00 UTC,-73.997710,40.756485,-73.987518,40.721337,1
253,2015-02-14 19:00:23.0000006,2015-02-14 19:00:23 UTC,-73.961975,40.776112,-73.954849,40.774075,1
905,2009-01-19 10:38:00.00000050,2009-01-19 10:38:00 UTC,-73.987080,40.741545,-73.990558,40.736778,1


### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [10]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [11]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(BaseEstimator, TransformerMixin):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
    """

    def __init__(self,start_lat="pickup_latitude",start_lon="pickup_longitude",end_lat="dropoff_latitude",end_lon="dropoff_longitude"):
        self.start_lat=start_lat
        self.start_lon=start_lon
        self.end_lat=end_lat
        self.end_lon=end_lon
        # A COMPPLETER
    
    def fit(self, X, y=None):
        return self


    def transform(self, X, y=None):
        X["distance"] = haversine_vectorized(X, start_lat="pickup_latitude", start_lon="pickup_longitude",
                                    end_lat="dropoff_latitude", end_lon="dropoff_longitude")
        return X[['distance']]

In [12]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

,distance
216,1.116850
82,1.487035
446,4.001499
253,0.641366
905,0.605668


In [13]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """
    def __init__(self, time_column, time_zone_name='America/New_York'):
        self.time_column = time_column
        self.time_zone_name = time_zone_name

    def extract_time_features(self, X):
        timezone_name = self.time_zone_name
        time_column = self.time_column
        df = X.copy()
        df.index = pd.to_datetime(df[time_column])
        df.index = df.index.tz_convert(timezone_name)
        df["dow"] = df.index.weekday
        df["hour"] = df.index.hour
        df["month"] = df.index.month
        df["year"] = df.index.year        
        return df
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        return self.extract_time_features(X)[['dow', 'hour', 'month', 'year']].reset_index(drop=True)

In [14]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder('pickup_datetime')
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

,dow,hour,month,year
0,2,15,4,2009
1,4,4,6,2013
2,4,18,2,2010
3,5,14,2,2015
4,0,5,1,2009


###  Prepocessing pipeline

In [15]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# create distance pipeline dist_pipe
pipe_distance = Pipeline(steps=[('dist_trans', DistanceTransformer()),
                ('stdscaler', StandardScaler())])

# display distance pipeline
pipe_distance

Pipeline(steps=[('dist_trans', DistanceTransformer()),
                ('stdscaler', StandardScaler())])

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [17]:
from sklearn.preprocessing import OneHotEncoder
# create time pipeline time_pipe
pipe_time = Pipeline([
    ('time_features_create', TimeFeaturesEncoder('pickup_datetime')),
    ('time_features_ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))
])
# display time pipeline
pipe_time


Pipeline(steps=[('time_features_create',
                 TimeFeaturesEncoder(time_column='pickup_datetime')),
                ('time_features_ohe',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [22]:
from sklearn.compose import ColumnTransformer
# create preprocessing pipeline preproc_pipe
preprocessor = ColumnTransformer([
    ('distance', pipe_distance, ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']),
    ('time', pipe_time, ['pickup_datetime'])
])
# display preprocessing pipeline
preprocessor



ColumnTransformer(transformers=[('distance',
                                 Pipeline(steps=[('dist_trans',
                                                  DistanceTransformer()),
                                                 ('stdscaler',
                                                  StandardScaler())]),
                                 ['pickup_longitude', 'pickup_latitude',
                                  'dropoff_longitude', 'dropoff_latitude']),
                                ('time',
                                 Pipeline(steps=[('time_features_create',
                                                  TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                 ('time_features_ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['pickup_datetime'])])

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [23]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe
pipe_prepro = Pipeline([
    ('dist_and_time', preprocessor),
    ('LRegression', LinearRegression())
])

# display the pipeline with model
pipe_prepro

Pipeline(steps=[('dist_and_time',
                 ColumnTransformer(transformers=[('distance',
                                                  Pipeline(steps=[('dist_trans',
                                                                   DistanceTransformer()),
                                                                  ('stdscaler',
                                                                   StandardScaler())]),
                                                  ['pickup_longitude',
                                                   'pickup_latitude',
                                                   'dropoff_longitude',
                                                   'dropoff_latitude']),
                                                 ('time',
                                                  Pipeline(steps=[('time_features_create',
                                                                   TimeFeaturesEncoder(time_column='pickup_datetime')),
                                                                  ('time_features_ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['pickup_datetime'])])),
                ('LRegression', LinearRegression())])

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [17]:
# train the pipelined model
model = pipe_prepro.fit(X_train, y_train)

# compute y_pred on the test set
y_pred=model.predict(X_test)

Use the RMSE to evaluate the performance of the model:

In [18]:
def compute_rmse(y_pred, y_true):
    
    return # A COMPLETER

In [19]:
# call compute_rmse
compute_rmse(y_pred, y_test)

10.175329127383847

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [25]:
# implement get_data() function
def get_data(nrows=10000):
    '''returns a DataFrame with nrows from s3 bucket'''
    # A COMPLETER
    return df

In [26]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    # A COMPLETER
    return df

In [27]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
     # A  COMPLETER
    return pipe

In [28]:
# implement train() function
def train(X_train, y_train, pipeline):
    '''returns a trained pipelined model'''
   # A COMPLETER
    return pipeline

In [29]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    # A COMPLETER
    print(rmse)
    return rmse

### Test the complete worflow

Use the above functions to test the complete workflow.

In [30]:
# store the data in a DataFrame
df = get_data()

# set X and y
y = df["fare_amount"]
X = df.drop("fare_amount", axis=1)

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

9.396264153241109


### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀